## <font color=red> Predict House Price Using Advanced Regression

In [ ]:
!pip install xgboost

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
pd.set_option("display.max_columns",100)

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge,Lasso
from sklearn.linear_model import ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.tree import ExtraTreeRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error


In [ ]:
def algo_test(x,y):
        #Bütün modelleri tanımlıyorum
        L=LinearRegression()
        R=Ridge()
        Lass=Lasso()
        E=ElasticNet()
        ETR=ExtraTreeRegressor()
        GBR=GradientBoostingRegressor()
        kn=KNeighborsRegressor()
        dt=DecisionTreeRegressor()
        xgb=XGBRegressor()
       
        
        algos=[L,R,Lass,E,ETR,GBR,kn,dt,xgb]
        algo_names=['Linear','Ridge','Lasso','ElasticNet','Extra Tree','Gradient Boosting','KNeighborsRegressor','Decision Tree','XGBRegressor']
        
        x_train, x_test, y_train, y_test=train_test_split(x,y,test_size=.2,random_state=42)
        
        r_squared= []
        rmse= []
        mae= []
        
        #Hata ve doğruluk oranlarını bir tablo haline getirmek için bir dataframe oluşturuyorum
        result=pd.DataFrame(columns=['R_Squared','RMSE','MAE'],index=algo_names)
        
        
        for algo in algos:
            p=algo.fit(x_train,y_train).predict(x_test)
            r_squared.append(r2_score(y_test,p))
            rmse.append(mean_squared_error(y_test,p)**.5)
            mae.append(mean_absolute_error(y_test,p))
        
            

        #result adlı tabloya doğruluk ve hata oranlarımı yerleştiriyorum
        result.R_Squared=r_squared
        result.RMSE=rmse
        result.MAE=mae
        
       #oluşturduğum result tablosunu doğruluk oranına (r2_score) göre sıralayıp dönüyor
        rtable=result.sort_values('R_Squared',ascending=False)
        return rtable

In [ ]:
df1=pd.read_csv('train.csv')

In [ ]:
df2=pd.read_csv('test.csv')

- Hedef olan sütunu direk y ye atıyorum.

In [ ]:
y=df1['SalePrice']

In [ ]:
abs(df1.corr()['SalePrice'].sort_values(ascending=False).head(22))

SalePrice       1.000000
OverallQual     0.790982
GrLivArea       0.708624
GarageCars      0.640409
GarageArea      0.623431
TotalBsmtSF     0.613581
1stFlrSF        0.605852
FullBath        0.560664
TotRmsAbvGrd    0.533723
YearBuilt       0.522897
YearRemodAdd    0.507101
GarageYrBlt     0.486362
MasVnrArea      0.477493
Fireplaces      0.466929
BsmtFinSF1      0.386420
LotFrontage     0.351799
WoodDeckSF      0.324413
2ndFlrSF        0.319334
OpenPorchSF     0.315856
HalfBath        0.284108
LotArea         0.263843
BsmtFullBath    0.227122
Name: SalePrice, dtype: float64

In [ ]:
df1.shape,df2.shape

((1460, 81), (1459, 80))

In [ ]:
df=df1.append(df2)

- df2 yani test verisinde saleprice sütunu yok eski çözümde o sütunu ekleyip sıfırla doldurmuştun sonrada tahmin ettiğin değerleri bu sütuna atmıştın ama ona hiç gerek yok en başya hedef sütuna salprice ı eşitledik artık silebiliriz böylece veriler birleştiğinde yani data da 1459 boş saleprice satırım olmayacak.

In [ ]:
del df["SalePrice"]

- Object olan ve sayı değerine sahip olan sütunları ayrı ayrı görmek istiyorum çünkü object olanlardaki verileri None ile dolduruken,diğerlerini 0 veya 1 ile dolduracağım.

In [ ]:
df[df.select_dtypes(include=['object']).columns]

,MSZoning,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinType2,Heating,HeatingQC,CentralAir,Electrical,KitchenQual,Functional,FireplaceQu,GarageType,GarageFinish,GarageQual,GarageCond,PavedDrive,PoolQC,Fence,MiscFeature,SaleType,SaleCondition
0,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,No,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,NaN,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
1,RL,Pave,NaN,Reg,Lvl,AllPub,FR2,Gtl,Veenker,Feedr,Norm,1Fam,1Story,Gable,CompShg,MetalSd,MetalSd,None,TA,TA,CBlock,Gd,TA,Gd,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
2,RL,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,CollgCr,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Mn,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
3,RL,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,Crawfor,Norm,Norm,1Fam,2Story,Gable,CompShg,Wd Sdng,Wd Shng,None,TA,TA,BrkTil,TA,Gd,No,ALQ,Unf,GasA,Gd,Y,SBrkr,Gd,Typ,Gd,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
4,RL,Pave,NaN,IR1,Lvl,AllPub,FR2,Gtl,NoRidge,Norm,Norm,1Fam,2Story,Gable,CompShg,VinylSd,VinylSd,BrkFace,Gd,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,Ex,Y,SBrkr,Gd,Typ,TA,Attchd,RFn,TA,TA,Y,NaN,NaN,NaN,WD,Normal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,Twnhs,2Story,Gable,CompShg,CemntBd,CmentBd,None,TA,TA,CBlock,TA,TA,No,Unf,Unf,GasA,Gd,Y,SBrkr,TA,Typ,NaN,NaN,NaN,NaN,NaN,Y,NaN,NaN,NaN,WD,Normal
1455,RM,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,MeadowV,Norm,Norm,TwnhsE,2Story,Gable,CompShg,CemntBd,CmentBd,None,TA,TA,CBlock,TA,TA,No,Rec,Unf,GasA,TA,Y,SBrkr,TA,Typ,NaN,CarPort,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1456,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,Gable,CompShg,VinylSd,VinylSd,None,TA,TA,CBlock,TA,TA,No,ALQ,Unf,GasA,Ex,Y,SBrkr,TA,Typ,TA,Detchd,Unf,TA,TA,Y,NaN,NaN,NaN,WD,Abnorml
1457,RL,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,SFoyer,Gable,CompShg,HdBoard,Wd Shng,None,TA,TA,PConc,Gd,TA,Av,GLQ,Unf,GasA,TA,Y,SBrkr,TA,Typ,NaN,NaN,NaN,NaN,NaN,Y,NaN,MnPrv,Shed,WD,Normal


In [ ]:
df[df.select_dtypes(include=[np.number]).columns]

,Id,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold
0,1,60,65.0,8450,7,5,2003,2003,196.0,706.0,0.0,150.0,856.0,856,854,0,1710,1.0,0.0,2,1,3,1,8,0,2003.0,2.0,548.0,0,61,0,0,0,0,0,2,2008
1,2,20,80.0,9600,6,8,1976,1976,0.0,978.0,0.0,284.0,1262.0,1262,0,0,1262,0.0,1.0,2,0,3,1,6,1,1976.0,2.0,460.0,298,0,0,0,0,0,0,5,2007
2,3,60,68.0,11250,7,5,2001,2002,162.0,486.0,0.0,434.0,920.0,920,866,0,1786,1.0,0.0,2,1,3,1,6,1,2001.0,2.0,608.0,0,42,0,0,0,0,0,9,2008
3,4,70,60.0,9550,7,5,1915,1970,0.0,216.0,0.0,540.0,756.0,961,756,0,1717,1.0,0.0,1,0,3,1,7,1,1998.0,3.0,642.0,0,35,272,0,0,0,0,2,2006
4,5,60,84.0,14260,8,5,2000,2000,350.0,655.0,0.0,490.0,1145.0,1145,1053,0,2198,1.0,0.0,2,1,4,1,9,1,2000.0,3.0,836.0,192,84,0,0,0,0,0,12,2008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1454,2915,160,21.0,1936,4,7,1970,1970,0.0,0.0,0.0,546.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,5,0,NaN,0.0,0.0,0,0,0,0,0,0,0,6,2006
1455,2916,160,21.0,1894,4,5,1970,1970,0.0,252.0,0.0,294.0,546.0,546,546,0,1092,0.0,0.0,1,1,3,1,6,0,1970.0,1.0,286.0,0,24,0,0,0,0,0,4,2006
1456,2917,20,160.0,20000,5,7,1960,1996,0.0,1224.0,0.0,0.0,1224.0,1224,0,0,1224,1.0,0.0,1,0,4,1,7,1,1960.0,2.0,576.0,474,0,0,0,0,0,0,9,2006
1457,2918,85,62.0,10441,5,5,1992,1992,0.0,337.0,0.0,575.0,912.0,970,0,0,970,0.0,1.0,1,0,3,1,6,0,NaN,0.0,0.0,80,32,0,0,0,0,700,7,2006


- from sklearn.impute import SimpleImputer
- imputer = SimpleImputer(strategy="median")
- data3=imputer.fit_transform(data3)
- data3=pd.DataFrame(data3,columns=idx)

  -  Bu kodu yazarsan eğer her sütundaki boş verileri o sütunun median ı ile doldurmuş olursun bu normalde her sütunu doldurmakla uğraşmak yerine kolay yoldur ancak burada bunu yaparsan o özelliğe sahip olmayan evlere o özelliği vermiş olursun.Mesela havuzu yada bodrum katı olmayan evlere median ile bir değer atadığın an o evlerde bu özellikler varmış gibi olur bu doğru olmaz.

In [ ]:
empty=df.isnull().sum()
empty[empty>0]

MSZoning           4
LotFrontage      486
Alley           2721
Utilities          2
Exterior1st        1
Exterior2nd        1
MasVnrType        24
MasVnrArea        23
BsmtQual          81
BsmtCond          82
BsmtExposure      82
BsmtFinType1      79
BsmtFinSF1         1
BsmtFinType2      80
BsmtFinSF2         1
BsmtUnfSF          1
TotalBsmtSF        1
Electrical         1
BsmtFullBath       2
BsmtHalfBath       2
KitchenQual        1
Functional         2
FireplaceQu     1420
GarageType       157
GarageYrBlt      159
GarageFinish     159
GarageCars         1
GarageArea         1
GarageQual       159
GarageCond       159
PoolQC          2909
Fence           2348
MiscFeature     2814
SaleType           1
dtype: int64

- Hangi sütunlarda boş veriler var ve ne kadar boş veri var görmek istiyorum.

In [ ]:
df.sample(10)

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,LandSlope,Neighborhood,Condition1,Condition2,BldgType,HouseStyle,OverallQual,OverallCond,YearBuilt,YearRemodAdd,RoofStyle,RoofMatl,Exterior1st,Exterior2nd,MasVnrType,MasVnrArea,ExterQual,ExterCond,Foundation,BsmtQual,BsmtCond,BsmtExposure,BsmtFinType1,BsmtFinSF1,BsmtFinType2,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,Heating,HeatingQC,CentralAir,Electrical,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,KitchenQual,TotRmsAbvGrd,Functional,Fireplaces,FireplaceQu,GarageType,GarageYrBlt,GarageFinish,GarageCars,GarageArea,GarageQual,GarageCond,PavedDrive,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition
946,947,80,RL,70.0,8163,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Norm,Norm,1Fam,SLvl,5,6,1959,1959,Gable,CompShg,HdBoard,HdBoard,BrkFace,128.0,TA,Gd,CBlock,TA,TA,Av,ALQ,748.0,BLQ,294.0,102.0,1144.0,GasA,TA,Y,SBrkr,1144,0,0,1144,1.0,0.0,1,0,3,1,TA,6,Typ,1,TA,Attchd,1959.0,RFn,1.0,796.0,TA,TA,Y,86,0,0,0,0,0,NaN,NaN,NaN,0,3,2006,WD,Normal
1092,1093,50,RL,60.0,8400,Pave,NaN,Reg,Bnk,AllPub,Inside,Gtl,SWISU,Norm,Norm,1Fam,1.5Fin,6,5,1925,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,PConc,TA,TA,No,Rec,423.0,Unf,0.0,758.0,1181.0,GasA,Fa,Y,SBrkr,1390,304,0,1694,0.0,0.0,2,0,4,1,TA,7,Typ,1,Gd,Detchd,1925.0,Unf,2.0,576.0,TA,TA,Y,342,0,128,0,0,0,NaN,NaN,NaN,0,6,2008,WD,Normal
201,202,20,RL,75.0,10125,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Norm,Norm,1Fam,1Story,6,6,1977,1977,Gable,CompShg,Plywood,Plywood,None,0.0,TA,TA,CBlock,TA,TA,No,ALQ,641.0,LwQ,279.0,276.0,1196.0,GasA,TA,Y,SBrkr,1279,0,0,1279,0.0,1.0,2,0,3,1,TA,6,Typ,2,Fa,Detchd,1980.0,Unf,2.0,473.0,TA,TA,Y,238,83,0,0,0,0,NaN,MnPrv,NaN,0,2,2008,WD,Normal
473,1934,60,RL,72.0,8702,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,6,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,TA,TA,No,BLQ,706.0,Unf,0.0,220.0,926.0,GasA,Ex,Y,SBrkr,926,678,0,1604,0.0,0.0,2,1,3,1,TA,7,Typ,1,TA,Attchd,1997.0,Fin,2.0,470.0,TA,TA,Y,0,36,0,0,0,0,NaN,NaN,NaN,0,4,2008,WD,Normal
418,419,50,RL,60.0,8160,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Edwards,Norm,Norm,1Fam,1.5Fin,5,6,1940,1950,Gable,CompShg,MetalSd,MetalSd,None,0.0,TA,TA,BrkTil,TA,TA,No,ALQ,312.0,Unf,0.0,444.0,756.0,GasA,Fa,N,FuseF,756,378,0,1134,1.0,0.0,1,1,3,1,TA,7,Typ,0,NaN,Detchd,1940.0,Unf,1.0,240.0,TA,TA,P,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2007,WD,AdjLand
613,614,20,RL,70.0,8402,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Mitchel,Feedr,Norm,1Fam,1Story,5,5,2007,2007,Gable,CompShg,VinylSd,VinylSd,None,0.0,TA,TA,PConc,Gd,TA,No,ALQ,206.0,Unf,0.0,914.0,1120.0,GasA,Ex,Y,SBrkr,1120,0,0,1120,0.0,0.0,1,0,3,1,TA,6,Typ,0,NaN,NaN,NaN,NaN,0.0,0.0,NaN,NaN,Y,0,30,0,0,0,0,NaN,NaN,NaN,0,12,2007,New,Partial
864,2325,20,RL,59.0,10936,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,1Story,7,5,2006,2006,Gable,CompShg,VinylSd,VinylSd,None,0.0,Gd,TA,PConc,Gd,TA,Av,Unf,0.0,Unf,0.0,1510.0,1510.0,GasA,Ex,Y,SBrkr,1525,0,0,1525,0.0,0.0,2,1,3,1,Gd,6,Typ,1,Gd,Attchd,2006.0,RFn,2.0,534.0,TA,TA,Y,100,18,0,0,0,0,NaN,NaN,NaN,0,4,2007,New,Partial
983,2444,50,RM,120.0,9000,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,OldTown,Norm,Norm,1Fam,1.5Fin,5,8,1900,2006,Gable,CompShg,Stucco,Stucco,None,0.0,TA,TA,BrkTil,TA,Fa,No,BLQ,130.0,Unf,0.0,550.0,680.0,GasA,Ex,Y,SBrkr,680,494,0,1174,0.0,0.0,1,0,2,1,Gd,6,Typ,1,Gd,Detchd,2000.0,Unf,2.0,576.0,TA,TA,Y,116,26,40,0,0,0,NaN,MnPrv,NaN,0,6,2007,WD,Normal
107,1568,60,RL,NaN,17082,Pave,NaN,IR1,Low,AllPub,CulDSac,Mod,ClearCr,Norm,Norm,1Fam,2Story,6,5,1978,1992,Gable,CompShg,VinylSd,VinylSd,BrkFace,288.0,TA,TA,PConc,Gd,TA,Av,ALQ,964.0,Unf,0.0,153.0,1117.0,GasA,Ex,Y,SBrkr,1117,864,0,1981,1.0,0.0,2,1,4,1,Gd,8,Typ,1,TA,Attchd,1978.0,Fin,2.0,522.0,TA,TA,Y,336,104,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal
669,670,30,RL,80.0,11600,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,Crawfor,Norm,Norm,1Fam,1Story,4,5,1922,1950,Ga

- Peki ben evlere olmayan özelliklerini kazandırmadan boş verileri nasıl dolduracağım??
  - Kategori olan sütunların boş verilerini None ile dolduracağım.
    - Allay,PoolQC,MiscFeature,Alley,Fence,FireplaceQu :mesela bu sütunlara çoğunda NaN yazıyor bu evlerde havuz,şömine,çit vb.olmadığını görüyoruz ve bu bilgileri ortalama bir değer ile doldurup şöminesi olmayan bir eve şöminesi var demek istemiyoruz bu sebeple bu category sütunlarının boş verilerini None ile dolduruyoruz. 

- df["PoolQC"] = df["PoolQC"].fillna("None")
- df["MiscFeature"] = df["MiscFeature"].fillna("None")
- df["Alley"] = df["Alley"].fillna("None")
- df["Fence"] = df["Fence"].fillna("None")
- df["FireplaceQu"] = df["FireplaceQu"].fillna("None")
  - Bu yolla da aşağıdaki yollada doldurabilirsin ikisinide öğren çünkü bir projede biri daha uygunken diğerinde diğeri daha uygun olabilir.Ama bir programcı aynı yada benzer satır kodları tekrar tekrar yazmaz onun bir kolay yolunu bulmaya çalışır.Aşağıdaki 2 satırlık kod gibi.

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond','PoolQC','MiscFeature','Alley','Fence','FireplaceQu'):
    df[col] = df[col].fillna('None')

- LotFrontage:Evin caddeye olan uzaklığı diyebiliriz.Peki bunları nasıl dolduracağız?Burada ki boş verileri None ile doldurmayacağız çünkü sütun object değil.Ve biz şunu biliyoruz ki bu veriler boş bile olsa her evin caddeye belli bir uzaklığı vardır ve filmlerde görüyoruz ki çoğu ev belli bir düzen ile inşa edilmiş.
- Bu durumda biz boş olan bu ev-cadde arası uzaklıkları orta bir değer ile doldurabiliriz.

In [ ]:
df["LotFrontage"] = df.groupby("Neighborhood")["LotFrontage"].transform(lambda x: x.fillna(x.median()))

- lambda tek kullanımlık fonksiyondur tek satır ile tüm boş verileri lambda ile doldurabiliriz.

- Peki sıra geldi numeric olan sütunların boş verilerini nasıl dolduracağız.Bunu yapmanın en zekice yolu hiçbir eve ekstra özellik kazandırmadan o evde bu özelliğin olmadığını belirtmektir.Ve bunu da boş verilerin yerine 0 vererek yapıyoruz.Sonuçta garaj alanı boşsa biz bunu garajı yok şeklinde ele alabiliriz.Bu nedenle 0 vermemiz sorun olmaz.
- GarageYrBlt,GarageArea,GarageCars,BsmtFinSF1, BsmtFinSF2, BsmtUnfSF, TotalBsmtSF, BsmtFullBath,BsmtHalfBath sütunlarında bodrum veya garaj yok şeklinde dolduracağız.

In [ ]:
for col in ('GarageYrBlt', 'GarageArea', 'GarageCars','BsmtFinSF1', 'BsmtFinSF2', 'BsmtUnfSF','TotalBsmtSF', 'BsmtFullBath', 'BsmtHalfBath'):
    df[col] = df[col].fillna(0)

- BsmtQual, BsmtCond, BsmtExposure, BsmtFinType1,BsmtFinType2 bu sütunlar ise bodrumla ilgili kategorik özelliklere sahip olan sütunlardır.Daha önced deiğim gibi bunlarıda None ile dolduracağız.

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    df[col] = df[col].fillna('None')

- MasVnrArea:duvar kağıdı alan 0 ile,MasVnrType:duvar kağıdı tipi ise None ile doldurulur.
- Not:Bazen sütunların ne ifade ettiğini anlayamayabilirsin(kısaltılmış şeklide yazıldığından vs.) bu durumlarda object ve numerical sütundan hangisi olduğuna bakıp ona göre None veya 0 ile doldurabilirsin.

In [ ]:
df["MasVnrType"] = df["MasVnrType"].fillna("None")
df["MasVnrArea"] = df["MasVnrArea"].fillna(0)

- MSZoning (The general zoning classification) :İmar , bir belediyenin veya başka bir hükümet kademesinin araziyi, her biri diğer bölgelerden farklı yeni gelişim için bir dizi düzenlemeye sahip olan bölgeler olarak adlandırılan alanlara böldüğü bir şehir planlama yöntemidir .Bu sütunu en yaygın olan değer ile dolduracağız.

In [ ]:
df['MSZoning']=df['MSZoning'].fillna(df['MSZoning'].mode()[0])

- [0] aynı sütundaki mode değerini bulmasını istediğimiz için yazdık.

- functional evin işlevselliğinin bir göstergesi olan bir sütundur.Kategorik bir sütundur ancak bunu None ile doldurmak eve bir fonksiyonu yok demek gibi bir şey olur bu yüzden böyle demektense bu sütunu da en yaygın olan değer ile dolduracağız.
- NOT:Sadece 1,2 gibi çok az sayıda boş veriye sahip sütunları o sütunun mode değeri ile doldurabiliriz.Mode fonk. güzel yanı hem kategorik hem sayısal sütunları doldurmak için kullanılabiliyor olmasıdır.

In [ ]:
for col in ('Functional','Electrical','KitchenQual','Exterior1st','Exterior2nd','SaleType','Utilities'):
    df[col] = df[col].fillna(df[col].mode()[0])

- MSSubClass:"Satışa dahil olan mesken tipi" olarak tanımlanır, sayısal olarak kodlanmıştır, ancak gerçekte kategorik bir değişkendir.(zipcode gibi sayılar barındırsada biz bu sütuna kategorik muamelesi yaparız.)
  - Burada dikkat etmemiz gereken en önemli şeylerden biri de bir sütun sayısal değerlere sahip diye o sütunun boş verilerini direk 0 ile dolduramayız.O sütun hakkında bir bilgimiz yoksa önce bir araştırma yapıp neyi temsil ettiğini anlamalıyız.
- Kategorik bir sütun olduğundan None ile dolduracağız.  

In [ ]:
df['MSSubClass'] = df['MSSubClass'].fillna("None")

In [ ]:
df.isnull().sum()

Id               0
MSSubClass       0
MSZoning         0
LotFrontage      0
LotArea          0
                ..
MiscVal          0
MoSold           0
YrSold           0
SaleType         0
SaleCondition    0
Length: 80, dtype: int64

In [ ]:
empty=df.isnull().sum()
empty[empty>0]

Series([], dtype: int64)

- Artık hiç boş verimizin olmadığını hepsini uygun şekilde doldurduğumuzu görüyoruz.

- zipcode da yaptığımız gibi bir evin diğerine üstünlüğünü ifade etmeyen,sayısal verilerle ifade edilen,sütunları kategori türüne dönüştürüp dummie uygulayacağız.

In [ ]:
df['MSSubClass'] = df['MSSubClass'].apply(str)
df['OverallCond'] = df['OverallCond'].astype(str)
df['YrSold'] = df['YrSold'].astype(str)
df['MoSold'] = df['MoSold'].astype(str)

In [ ]:
df3=pd.get_dummies(df,drop_first=True)
df3.sample(15)

,Id,LotFrontage,LotArea,OverallQual,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,TotalBsmtSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_150,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSSubClass_20,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSZoning_FV,MSZoning_RH,...,GarageQual_Gd,GarageQual_None,GarageQual_Po,GarageQual_TA,GarageCond_Fa,GarageCond_Gd,GarageCond_None,GarageCond_Po,GarageCond_TA,PavedDrive_P,PavedDrive_Y,PoolQC_Fa,PoolQC_Gd,PoolQC_None,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_None,MiscFeature_None,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,MoSold_10,MoSold_11,MoSold_12,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
1080,2541,64.0,8333,7,2006,2006,0.0,0.0,0.0,738.0,738.0,738,753,0,1491,0.0,0.0,2,1,3,1,7,1,2006.0,2.0,484.0,100,30,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
638,639,67.0,8777,5,1910,1950,0.0,0.0,0.0,796.0,796.0,796,0,0,796,0.0,0.0,1,0,2,1,4,0,0.0,0.0,0.0,328,0,164,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
403,404,93.0,12090,8,1998,1998,650.0,0.0,0.0,1141.0,1141.0,1165,1098,0,2263,0.0,0.0,2,1,4,1,10,1,1998.0,2.0,420.0,144,123,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
438,439,40.0,4280,5,1913,2002,0.0,365.0,0.0,75.0,440.0,694,0,0,694,0.0,0.0,1,0,2,1,4,1,1990.0,1.0,352.0,0,0,34,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,1,1,0,0,0,1,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
1120,2581,65.0,6565,4,1957,1980,0.0,967.0,0.0,106.0,1073.0,1073,0,0,1073,1.0,0.0,1,0,3,1,6,0,1985.0,2.0,720.0,0,444,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
1247,1248,74.0,12328,6,1976,1976,335.0,539.0,0.0,473.0,1012.0,1034,0,0,1034,1.0,0.0,1,0,3,1,6,0,1976.0,3.0,888.0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0
538,539,72.0,11553,5,1968,1968,188.0,673.0,0.0,378.0,1051.0,1159,0,0,1159,0.0,0.0,1,1,3,1,7,1,1968.0,1.0,336.0,466,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0
870,871,60.0,6600,5,1962,1962,0.0,0.0,0.0,894.0,894.0,894,0,0,894,0.0,0.0,1,0,2,1,5,0,1962.0,1.0,308.0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0
117,118,74.0,8536,5,2006,2007,0.0,0.0,0.0,1125.0,1125.0,1125,0,0,1125,0.0,0.0,1,1,2,1,5,0,2007.0,2.0,430.0,80,64,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
830,2291,110.0,13440,8,2006,2007,190.0,0.0,0.0,1108.0,1108.0,1148,1402,0,2550,0.0,0.0,2,1,4,1,11,1,2006.0,3.0,670.0,120,39,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,0,1,0,0,1,0,0,0,1,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1


- Id nin bir evin fiyatını belirlemesinde hiçbir etkisinin olmadığını biliyoruz bu nedenle tahmin ederken de dikkate almak istemediğimizden sütunu siliyoruz.

In [ ]:
del df3['Id']

In [ ]:
df1.shape,df2.shape

((1460, 81), (1459, 80))

In [ ]:
df12=df3[:1460]
df22=df3[1460:]

In [ ]:
algo_test(df12,y)

,R_Squared,RMSE,MAE
Gradient Boosting,0.895958,28249.537173,17819.357670
Ridge,0.876471,30781.630848,20142.826244
XGBRegressor,0.875698,30877.757974,17841.050768
Lasso,0.840265,35003.172611,19448.488237
ElasticNet,0.833726,35712.418498,20981.323071
Decision Tree,0.824280,36712.837231,24415.945205
Extra Tree,0.729246,45571.600576,29438.880137
KNeighborsRegressor,0.699279,48027.365988,28554.471918
Linear,-0.039790,89305.796088,24526.087926


- En yüksek başarı oranına sahip olan olduğumuz model ile tahminlerimizi yapıp kaydedeceğiz.
  - Bunun için öncelikle boş bir dataframe (tablo) oluşturup feature engineering ile Id ve SalePrice sütunları açacağız.(Çünkü amacım hangi Id numarasına sahip olan ev ne kadarlık bir fiyata sahip olacak onu görmek.)Daha sonra da tahminlerimizi buradaki SalePrice sütununa atacağız.Ve bu dosyayı csv olarak kaydedeceğiz.

In [ ]:
df12_train, df12_test, y_train, y_test=train_test_split(df12,y,test_size=.2,random_state=42)
gb=GradientBoostingRegressor()
pred=gb.fit(df12_train,y_train).predict(df22)

In [ ]:
predictions=pd.DataFrame()
predictions['Id']=df2['Id']
predictions['SalePrice']=pred
predictions.to_csv('predictions.csv',index=False)
#index=False dememizin nedeni 0 dan başlayıp her satır sıralanan indexlerin kaydettiğim dosyada olmasını istememem.